In [1]:
from seeq import spy

In [2]:
spy.login(username='Your Username Here', password='Your Password Here', url='Server URL')

In [3]:
from seeq.spy.assets import Asset

class HVAC_Monitoring_Requirements(Asset):

    @Asset.Requirement()
    def Compressor_Power(self, metadata):
        """
        Compressor Power as measured at the power supply of the equipment
        itself.
        """
        return {
            'Type': 'Signal',
            'Unit Of Measure Family': 'kW'
        }

    @Asset.Requirement()
    def Temperature(self, metadata):
        """
        The ambient *external* temperature as measured in a neutral location
        away from HVAC inlet/outlet sites.

        > Note that this value may be derived from equipment sensors with a
        > suitable coefficient to account for location bias.
        """
        return {
            'Type': 'Signal',
            'Unit Of Measure Family': 'F',
            'Optional': True
        }

In [4]:
class HVAC_Monitoring(HVAC_Monitoring_Requirements):

    @Asset.Attribute()
    def Inefficient_Operation(self, metadata):
        """
        A condition that is present whenever the HVAC is running in ostensibly
        unnecessary environmental situations, resulting in inefficient use of power.

        Examples:

        * Ambient temperature is within control bounds and does not warrant
          cooling
        * Environmental conditions are such that HVAC system is operating outside
          of its range of capabilities
        """

        if not self.Temperature():
            return None

        return {
            'Type': 'Condition',
            'Formula Parameters': {
                '$power': self.Compressor_Power(),
                '$temp': self.Temperature()
            },
            'Formula': '$power > 20 kW and $temp < 60 F'
        }

In [12]:
# Grab the whole tree as the basis of the metadata DataFrame
metadata_df = spy.search({
    'Path': 'Example'
}, recursive=True)

,Path,Time,Count,Pages,Result
0,Example,00:00:01.38,75,1,Success


In [13]:
metadata_df

,ID,Path,Asset,Name,Description,Type,Value Unit Of Measure,Datasource Name,Archived
0,4C1CB882-40AF-441C-BB15-2D0D8E0F991B,NaN,Example,Cooling Tower 2,NaN,Asset,NaN,Example Data,False
1,00F97575-EDB4-4BA5-916A-E67620A80527,Example,Cooling Tower 2,Area E,NaN,Asset,NaN,Example Data,False
2,F83DCBAE-C347-4926-808A-3419D1562E89,Example >> Cooling Tower 2,Area E,Compressor Stage,NaN,StoredSignal,string,Example Data,False
3,21043814-6119-409B-93B4-DD7DE800BB7F,NaN,Example,Cooling Tower 1,NaN,Asset,NaN,Example Data,False
4,07585651-7400-4BB1-AE9D-6F8471AEC06B,Example,Cooling Tower 1,Area C,NaN,Asset,NaN,Example Data,False
...,...,...,...,...,...,...,...,...,...
70,25B5A0D8-8FB4-461C-AD58-A58A442D26BF,Example >> Cooling Tower 2,Area D,Wet Bulb,NaN,StoredSignal,°F,Example Data,False
71,1D9CC92F-C608-4C9E-99FD-E61092BAEE8E,Example >> Cooling Tower 2,Area D,Temperature,NaN,StoredSignal,°F,Example Data,False
72,9DFFC494-7C85-45C4-92E1-9814BC18C49E,Example >> Cooling Tower 2,Area D,Compressor Power,NaN,StoredSignal,kW,Example Data,False
73,A67666EB-229B-4710-9B88-03278B73FA04,Example >> Cooling Tower 2,Area D,Compressor Stage,NaN,StoredSignal,string,Example Data,False


In [5]:
# Grab the whole tree as the basis of the metadata DataFrame
metadata_df = spy.search({
    'Path': 'Example'
}, recursive=True)

# Reduce the number of columns (just for readability)
metadata_df = metadata_df[['ID', 'Path', 'Asset', 'Name', 'Type']]

# This call prepares the metadata DataFrame by adding 'Build Path' and
# 'Build Asset' columns. You can also specify a new root_asset_name, which
# is often desired so that you can differentiate the output tree from the
# input tree.
spy.assets.prepare(metadata_df, root_asset_name='HVAC Monitoring')

# Specify the Build Template. We want to target all leaf "Area" assets (for now)
metadata_df.at[metadata_df['Build Asset'].str.startswith('Area '), 'Build Template'] = 'HVAC_Monitoring'

metadata_df.head()

,Path,Time,Count,Pages,Result
0,Example,00:00:01.26,75,1,Success


,ID,Path,Asset,Name,Type,Build Path,Build Asset,Build Template
0,4C1CB882-40AF-441C-BB15-2D0D8E0F991B,NaN,Example,Cooling Tower 2,Asset,HVAC Monitoring,Cooling Tower 2,NaN
1,00F97575-EDB4-4BA5-916A-E67620A80527,Example,Cooling Tower 2,Area E,Asset,HVAC Monitoring >> Cooling Tower 2,Area E,HVAC_Monitoring
2,F83DCBAE-C347-4926-808A-3419D1562E89,Example >> Cooling Tower 2,Area E,Compressor Stage,StoredSignal,HVAC Monitoring >> Cooling Tower 2,Area E,HVAC_Monitoring
3,21043814-6119-409B-93B4-DD7DE800BB7F,NaN,Example,Cooling Tower 1,Asset,HVAC Monitoring,Cooling Tower 1,NaN
4,07585651-7400-4BB1-AE9D-6F8471AEC06B,Example,Cooling Tower 1,Area C,Asset,HVAC Monitoring >> Cooling Tower 1,Area C,HVAC_Monitoring


In [6]:
build_df = spy.assets.build(HVAC_Monitoring, metadata_df)

,Build Path,Build Asset,Build Template,Build Result
0,HVAC Monitoring >> Cooling Tower 2,Area E,HVAC_Monitoring,Success
1,HVAC Monitoring >> Cooling Tower 1,Area C,HVAC_Monitoring,Success
2,HVAC Monitoring >> Cooling Tower 1,Area G,HVAC_Monitoring,Success
3,HVAC Monitoring >> Cooling Tower 1,Area J,HVAC_Monitoring,Success
4,HVAC Monitoring >> Cooling Tower 1,Area I,HVAC_Monitoring,Success
5,HVAC Monitoring >> Cooling Tower 1,Area B,HVAC_Monitoring,Success
6,HVAC Monitoring >> Cooling Tower 1,Area A,HVAC_Monitoring,Success
7,HVAC Monitoring >> Cooling Tower 1,Area K,HVAC_Monitoring,Success
8,HVAC Monitoring >> Cooling Tower 1,Area H,HVAC_Monitoring,Success
9,HVAC Monitoring >> Cooling Tower 2,Area F,HVAC_Monitoring,Success


In [7]:
push_results_df = spy.push(metadata=build_df, quiet=True)

In [8]:
html = spy.assets.brochure(HVAC_Monitoring, output='html')

# Write it to a file so we can view it in the browser
with open('hvac_monitoring_brochure.html', 'w') as f:
    f.write(html)